# Libraries

In [1]:
import textract
import os
from collections import defaultdict
from pathlib import Path
import pandas as pd
import swifter
import string
import nltk
from os import listdir
from os.path import isfile, join
from PyPDF2 import PdfFileReader
from pdfminer.high_level import extract_text
from unicodedata import normalize
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
try:
    from PIL import Image
except ImportError:
    import Image
import cv2
import pytesseract
from pdf2image import convert_from_bytes
import numpy as np
import pandas as pd
import nltk
import re
from bs4 import BeautifulSoup
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
from nltk.corpus import wordnet
import unicodedata
from gensim import corpora
from six import iteritems
from nltk.corpus import names
from nltk.corpus import wordnet

# Functions

In [2]:
def extract_text(file_name, directory):
    try:
        text = textract.process(os.path.join(directory, file_name)).decode('utf-8')
        if len(text) < 200:
            print(file_name)
            print('Trying OCR')
            
            text = get_text_from_ocr(os.path.join(directory, file_name))
            print('done')
    except:
        
        print(file_name)
        print('Unsuccesful, trying OCR')
        text = get_text_from_ocr(os.path.join(directory, file_name))
        
    return(text)

In [3]:
def get_conf(page_gray):
    '''return a average confidence value of OCR result '''
    df = pytesseract.image_to_data(page_gray,output_type='data.frame')
    df.drop(df[df.conf==-1].index.values,inplace=True)
    df.reset_index()
    return df.conf.mean()
  
def deskew(image):
    '''deskew the image'''
    gray = cv2.bitwise_not(image)
    temp_arr = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    coords = np.column_stack(np.where(temp_arr > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

def get_text_from_ocr(file_with_dir):
    pdf_file = convert_from_bytes(open(file_with_dir, 'rb').read())
    text = ''
    for (i,page) in enumerate(pdf_file) :
        try:
            # transfer image of pdf_file into array
            page_arr = np.asarray(page)
            # transfer into grayscale
            page_arr_gray = cv2.cvtColor(page_arr,cv2.COLOR_BGR2GRAY)
            # deskew the page
            page_deskew = deskew(page_arr_gray)
            # cal confidence value
            page_conf = get_conf(page_deskew)
            # extract string 
            text += pytesseract.image_to_string(page_deskew)
                
        except:
            continue
    
    return(text)

# Read Files

In [4]:
os.getcwd()

'/home/alex/Desktop/DataMining/flex-mining'

In [5]:
dir_pdfs = '../Original_Papers/Original_Papers' 


In [6]:
files = [f for f in listdir(dir_pdfs) if isfile(join(dir_pdfs, f))]
files.sort()
files

['Abarbanell and Bushee 1998 AR.pdf',
 'Acharya Pedersen 2005 JFE.pdf',
 'Adrian Etula Muir 2014 JF.pdf',
 'Akbas Armstrong Petkova 2011 WP.pdf',
 'Ali Hwang Trombley 2003 JFE.pdf',
 'Alwathainani 2006 Dissertation.pdf',
 'Alwathainani 2009 BAR.pdf',
 'Amihud 2002 JFM.pdf',
 'Amihud and Mendelson 1986 JFE.pdf',
 'An Ang Bali Cakici 2014 JF.pdf',
 'Anderson Ghysels Juergens 2005 RFS.pdf',
 'Anderson Ghysels Juergens 2009 JFE.pdf',
 'Anderson and Garcia_Feijoo 2006 JF.pdf',
 'Ang Bali Cakici 2012 JF.pdf',
 'Ang Chen Xing 2001 WP.pdf',
 'Ang Chen Xing 2004 WP.pdf',
 'Ang Chen Xing 2006 RFS.pdf',
 'Ang Hodrick Xing Zhang 2006 JF.pdf',
 'Ang et al 2006 JF.pdf',
 'Asness Frazzini 2013 JPM.pdf',
 'Asness Frazzini Israel Moskowitz Pedersen 2018 JFE.pdf',
 'Asness Frazzini Pedersen 2013 WP.pdf',
 'Asness Frazzini Pedersen 2019 RAS.pdf',
 'Asquith Pathak and Ritter 2005 JFE.pdf',
 'Avramov Hordia Jostova and Philipov 2007 JF.pdf',
 'Baik Ahn 2007 Other.pdf',
 'Balakrishnan Bartov Faurel 2010 JAE

In [7]:
df = pd.DataFrame({'file_names':files})
df

,file_names
0,Abarbanell and Bushee 1998 AR.pdf
1,Acharya Pedersen 2005 JFE.pdf
2,Adrian Etula Muir 2014 JF.pdf
3,Akbas Armstrong Petkova 2011 WP.pdf
4,Ali Hwang Trombley 2003 JFE.pdf
...,...
218,Whited Wu 2006 RFS.pdf
219,Xie 2001 TAR.pdf
220,Xing Zhang and Zhao 2010 JFQA.pdf
221,Yan 2011 JFE.pdf


# Extract Text

In [8]:
df['text'] =  df.file_names.apply(extract_text, directory = dir_pdfs)

Watkins 2003.pdf
Trying OCR
done


In [9]:
df['text']

0      Abnormal Returns to a Fundamental Analysis Str...
1      ARTICLE IN PRESS\n\nJournal of Financial Econo...
2      THE JOURNAL OF FINANCE • VOL. LXIX, NO. 6 • DE...
3      The Volatility of Liquidity and Expected\nStoc...
4      Journal of Financial Economics 69 (2003) 355–3...
                             ...                        
218    Financial Constraints Risk\n\nGuojun Wu\nUnive...
219    The Mispricing of Abnormal Accruals\nAuthor(s)...
220    Vol. 45, No. 3, June 2010, pp. 641–662\n\ndoi:...
221    Journal of Financial Economics 99 (2011) 216–2...
222    THE JOURNAL OF FINANCE • VOL. LXI, NO. 1 • FEB...
Name: text, Length: 223, dtype: object

In [10]:
df

,file_names,text
0,Abarbanell and Bushee 1998 AR.pdf,Abnormal Returns to a Fundamental Analysis Str...
1,Acharya Pedersen 2005 JFE.pdf,ARTICLE IN PRESS\n\nJournal of Financial Econo...
2,Adrian Etula Muir 2014 JF.pdf,"THE JOURNAL OF FINANCE • VOL. LXIX, NO. 6 • DE..."
3,Akbas Armstrong Petkova 2011 WP.pdf,The Volatility of Liquidity and Expected\nStoc...
4,Ali Hwang Trombley 2003 JFE.pdf,Journal of Financial Economics 69 (2003) 355–3...
...,...,...
218,Whited Wu 2006 RFS.pdf,Financial Constraints Risk\n\nGuojun Wu\nUnive...
219,Xie 2001 TAR.pdf,The Mispricing of Abnormal Accruals\nAuthor(s)...
220,Xing Zhang and Zhao 2010 JFQA.pdf,"Vol. 45, No. 3, June 2010, pp. 641–662\n\ndoi:..."
221,Yan 2011 JFE.pdf,Journal of Financial Economics 99 (2011) 216–2...


In [11]:
df.to_pickle('../IntermediateText/text_with_pdf_name.pkl')

In [12]:
df = pd.read_pickle('../IntermediateText/text_with_pdf_name.pkl')

In [13]:
files = [f for f in listdir(dir_pdfs) if isfile(join(dir_pdfs, f))]
files.sort()
files

['Abarbanell and Bushee 1998 AR.pdf',
 'Acharya Pedersen 2005 JFE.pdf',
 'Adrian Etula Muir 2014 JF.pdf',
 'Akbas Armstrong Petkova 2011 WP.pdf',
 'Ali Hwang Trombley 2003 JFE.pdf',
 'Alwathainani 2006 Dissertation.pdf',
 'Alwathainani 2009 BAR.pdf',
 'Amihud 2002 JFM.pdf',
 'Amihud and Mendelson 1986 JFE.pdf',
 'An Ang Bali Cakici 2014 JF.pdf',
 'Anderson Ghysels Juergens 2005 RFS.pdf',
 'Anderson Ghysels Juergens 2009 JFE.pdf',
 'Anderson and Garcia_Feijoo 2006 JF.pdf',
 'Ang Bali Cakici 2012 JF.pdf',
 'Ang Chen Xing 2001 WP.pdf',
 'Ang Chen Xing 2004 WP.pdf',
 'Ang Chen Xing 2006 RFS.pdf',
 'Ang Hodrick Xing Zhang 2006 JF.pdf',
 'Ang et al 2006 JF.pdf',
 'Asness Frazzini 2013 JPM.pdf',
 'Asness Frazzini Israel Moskowitz Pedersen 2018 JFE.pdf',
 'Asness Frazzini Pedersen 2013 WP.pdf',
 'Asness Frazzini Pedersen 2019 RAS.pdf',
 'Asquith Pathak and Ritter 2005 JFE.pdf',
 'Avramov Hordia Jostova and Philipov 2007 JF.pdf',
 'Baik Ahn 2007 Other.pdf',
 'Balakrishnan Bartov Faurel 2010 JAE

In [14]:
df['new_file_name'] = files
df

,file_names,text,new_file_name
0,Abarbanell and Bushee 1998 AR.pdf,Abnormal Returns to a Fundamental Analysis Str...,Abarbanell and Bushee 1998 AR.pdf
1,Acharya Pedersen 2005 JFE.pdf,ARTICLE IN PRESS\n\nJournal of Financial Econo...,Acharya Pedersen 2005 JFE.pdf
2,Adrian Etula Muir 2014 JF.pdf,"THE JOURNAL OF FINANCE • VOL. LXIX, NO. 6 • DE...",Adrian Etula Muir 2014 JF.pdf
3,Akbas Armstrong Petkova 2011 WP.pdf,The Volatility of Liquidity and Expected\nStoc...,Akbas Armstrong Petkova 2011 WP.pdf
4,Ali Hwang Trombley 2003 JFE.pdf,Journal of Financial Economics 69 (2003) 355–3...,Ali Hwang Trombley 2003 JFE.pdf
...,...,...,...
218,Whited Wu 2006 RFS.pdf,Financial Constraints Risk\n\nGuojun Wu\nUnive...,Whited Wu 2006 RFS.pdf
219,Xie 2001 TAR.pdf,The Mispricing of Abnormal Accruals\nAuthor(s)...,Xie 2001 TAR.pdf
220,Xing Zhang and Zhao 2010 JFQA.pdf,"Vol. 45, No. 3, June 2010, pp. 641–662\n\ndoi:...",Xing Zhang and Zhao 2010 JFQA.pdf
221,Yan 2011 JFE.pdf,Journal of Financial Economics 99 (2011) 216–2...,Yan 2011 JFE.pdf


In [15]:
df['year'] = df.new_file_name.str.extract('(\d{4})')

In [16]:
df['author'] = df.new_file_name.str.extract('(.*) \d{4}')

In [17]:
df['journal'] = df.new_file_name.str.extract('\d{4} ([A-z]+)\s?.*\.')
df['signalname'] = df.new_file_name.str.extract('\d{4} [A-z]+\s(.*)\.')

In [18]:
df.head(35)

,file_names,text,new_file_name,year,author,journal,signalname
0,Abarbanell and Bushee 1998 AR.pdf,Abnormal Returns to a Fundamental Analysis Str...,Abarbanell and Bushee 1998 AR.pdf,1998,Abarbanell and Bushee,AR,NaN
1,Acharya Pedersen 2005 JFE.pdf,ARTICLE IN PRESS\n\nJournal of Financial Econo...,Acharya Pedersen 2005 JFE.pdf,2005,Acharya Pedersen,JFE,NaN
2,Adrian Etula Muir 2014 JF.pdf,"THE JOURNAL OF FINANCE • VOL. LXIX, NO. 6 • DE...",Adrian Etula Muir 2014 JF.pdf,2014,Adrian Etula Muir,JF,NaN
3,Akbas Armstrong Petkova 2011 WP.pdf,The Volatility of Liquidity and Expected\nStoc...,Akbas Armstrong Petkova 2011 WP.pdf,2011,Akbas Armstrong Petkova,WP,NaN
4,Ali Hwang Trombley 2003 JFE.pdf,Journal of Financial Economics 69 (2003) 355–3...,Ali Hwang Trombley 2003 JFE.pdf,2003,Ali Hwang Trombley,JFE,NaN
5,Alwathainani 2006 Dissertation.pdf,Virginia Commonwealth University\n\nVCU Schola...,Alwathainani 2006 Dissertation.pdf,2006,Alwathainani,Dissertation,NaN
6,Alwathainani 2009 BAR.pdf,The British Accounting Review 41 (2009) 184–19...,Alwathainani 2009 BAR.pdf,2009,Alwathainani,BAR,NaN
7,Amihud 2002 JFM.pdf,Journal of Financial Markets 5 (2002) 31–56\n\...,Amihud 2002 JFM.pdf,2002,Amihud,JFM,NaN
8,Amihud and Mendelson 1986 JFE.pdf,Journal\n\nof Financial\n\nEconomics\n\n17 (19...,Amihud and Mendelson 1986 JFE.pdf,1986,Amihud and Mendelson,JFE,NaN
9,An Ang Bali Cakici 2014 JF.pdf,"THE JOURNAL OF FINANCE • VOL. LXIX, NO. 5 • OC...",An Ang Bali Cakici 2014 JF.pdf,2014,An Ang Bali Cakici,JF,NaN


In [19]:
df[~df['signalname'].isna()]

,file_names,text,new_file_name,year,author,journal,signalname
108,Fama French 1997 JFE industry.pdf,I\n\nE1.5EVIER\n\nJOURNAL OF\n\nJournal of Fin...,Fama French 1997 JFE industry.pdf,1997,Fama French,JFE,industry
181,Novy-Marx 2010 RF OPLeverage.pdf,Review of Finance (2011) 15: 103–134\ndoi: 10....,Novy-Marx 2010 RF OPLeverage.pdf,2010,Novy-Marx,RF,OPLeverage
182,Novy-Marx 2012 JFE IntMom.pdf,Journal of Financial Economics 103 (2012) 429–...,Novy-Marx 2012 JFE IntMom.pdf,2012,Novy-Marx,JFE,IntMom
184,Novy-Marx 2013 JFE GP.pdf,Journal of Financial Economics 108 (2013) 1–28...,Novy-Marx 2013 JFE GP.pdf,2013,Novy-Marx,JFE,GP


In [20]:
df[df['journal'].isna()].head(50)

,file_names,text,new_file_name,year,author,journal,signalname
100,"Easley, Hvidkjaer and O'Hara 2002 - JF.pdf",Is Information Risk a Determinant\nof Asset Re...,"Easley, Hvidkjaer and O'Hara 2002 - JF.pdf",2002,"Easley, Hvidkjaer and O'Hara",NaN,NaN
189,Palacios Fama French Other.pdf,Fama-French Factors\n\n1 of 13\n\nhttps://wrds...,Palacios Fama French Other.pdf,NaN,NaN,NaN,NaN
210,Stambaugh Yu Yuan Appendix.pdf,Documentation: Mispricing Measure Data\nby\nRo...,Stambaugh Yu Yuan Appendix.pdf,NaN,NaN,NaN,NaN
211,Technical Analysis Manuscript.pdf,Technical Analysis and the Cross-Section of St...,Technical Analysis Manuscript.pdf,NaN,NaN,NaN,NaN
217,Watkins 2003.pdf,Niding the Wave of Sentiment: An Analysis Of I...,Watkins 2003.pdf,2003,Watkins,NaN,NaN


# Text Cleaning

In [21]:
df["text"] = df["text"].str.replace(r'\s*https?://\S+(\s+|$)', ' ')
df["text"] = df["text"].str.replace(r'[\w.+-]+@[\w-]+\.[\w.-]+', ' ')

In [22]:
df.to_csv('../IntermediateText/text_with_pdf_name.csv', index = False)